In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Loading the dataset
df = pd.read_csv(r'C:\Users\DELL\Desktop\electronics_shop_sales.csv')

# Converting 'sale_date' to datetime format
df['sale_date'] = pd.to_datetime(df['sale_date'])

# Adding 'Month' and 'Year' columns to make aggregation easier
df['Month'] = df['sale_date'].dt.month
df['Year'] = df['sale_date'].dt.year

# Aggregating sales by Product Name, Year, and Month
df_monthly_sales = df.groupby(['product_name', 'Year', 'Month']).agg({'quantity_sold': 'sum'}).reset_index()

# Creating a lag column for previous month's sales for each product
df_monthly_sales['Previous_Month_Sales'] = df_monthly_sales.groupby('product_name')['quantity_sold'].shift(1)

# Dropping rows where previous month's sales are missing (first month for each product)
df_monthly_sales = df_monthly_sales.dropna(subset=['Previous_Month_Sales'])

# Preparing features (X) and target (y)
X = df_monthly_sales[['Previous_Month_Sales']]  # Feature: previous month's sales
y = df_monthly_sales['product_name']  # Target: product name

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training Random Forest model
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train, y_train)

# Predicting on the test set
y_pred = rf.predict(X_test)

# Get unique predictions and count occurrences
unique_preds, counts = pd.Series(y_pred).value_counts().reset_index().values.T

# Function to determine priority based on predicted sales frequency
def get_priority(count, max_count):
    if count == max_count:
        return "High Priority"
    elif count >= max_count * 0.7:
        return "Medium Priority"
    else:
        return "Low Priority"

# Get the maximum count to determine relative priorities
max_count = max(counts)

# Format and print the predictions
print("\nRecommended Products for Next Month's Inventory:")
print("----------------------------------------------")

for product, count in zip(unique_preds, counts):
    priority = get_priority(count, max_count)
    print(f"\n{product}")
    print(priority)


Recommended Products for Next Month's Inventory:
----------------------------------------------

Smart Watch
High Priority

Laptop
Low Priority

Bluetooth Speaker
Low Priority

Washing Machine
Low Priority


C:\Users\DELL\AppData\Local\Temp\ipykernel_1472\3391585818.py:9: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['sale_date'] = pd.to_datetime(df['sale_date'])
